<a href="https://colab.research.google.com/github/jmcarbo/databases/blob/master/database_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy
import altair as alt
import numpy as np
import glob
import os

In [ ]:
!git clone https://github.com/CSSEGISandData/COVID-19.git

In [ ]:
!ls COVID-19/csse_covid_19_data/csse_covid_19_daily_reports

In [ ]:
path = r'COVID-19/csse_covid_19_data/csse_covid_19_daily_reports' # use your path
all_files = glob.glob(os.path.join(path, "*.csv"))

df = pd.concat((pd.read_csv(f).assign(Date=lambda df: os.path.basename(f)[:-4]) for f in all_files))
df.Date = pd.to_datetime(df.Date, format='%m-%d-%Y')

In [ ]:
df

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio,Date,Province/State,Country/Region,Last Update,Incidence_Rate,Case-Fatality_Ratio,Latitude,Longitude
0,NaN,NaN,NaN,Afghanistan,2020-12-24 05:23:04,33.93911,67.709953,50433.0,2117.0,39692.0,8624.0,Afghanistan,129.553427,4.197648,2020-12-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,Albania,2020-12-24 05:23:04,41.15330,20.168300,54317.0,1117.0,29799.0,23401.0,Albania,1887.448746,2.056446,2020-12-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,Algeria,2020-12-24 05:23:04,28.03390,1.659600,96549.0,2696.0,64401.0,29452.0,Algeria,220.174923,2.792364,2020-12-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,Andorra,2020-12-24 05:23:04,42.50630,1.521800,7669.0,82.0,7106.0,481.0,Andorra,9925.580793,1.069240,2020-12-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,Angola,2020-12-24 05:23:04,-11.20270,17.873900,16931.0,393.0,9729.0,6809.0,Angola,51.514824,2.321186,2020-12-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,2020-03-21,NaN,Jersey,2020-03-17T18:33:03,NaN,NaN,49.1900,-2.110
300,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,2020-03-21,NaN,Puerto Rico,2020-03-17T16:13:14,NaN,NaN,18.2000,-66.500
301,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,2020-03-21,NaN,Republic of the Congo,2020-03-17T21:33:03,NaN,NaN,-1.4400,15.556
302,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,2020-03-21,NaN,The Bahamas,2020-03-19T12:13:38,NaN,NaN,24.2500,-76.000


In [ ]:
df.Date.min()

Timestamp('2020-01-22 00:00:00')

In [ ]:
for index, row in df.iterrows():
  if pd.isna(row['Country_Region']):
    row['Country_Region']=row['Country/Region']

In [ ]:
df[pd.notnull(df.Incidence_Rate)][['Country_Region','Combined_Key','Date','Incidence_Rate','Confirmed']].assign(Population=lambda df: (df.Confirmed/df.Incidence_Rate)*100000)

In [ ]:
df['Country_Region']=np.where(pd.isnull(df.Country_Region),df['Country/Region'],df['Country_Region'])
df['Province_State']=np.where(pd.isnull(df['Province_State']),df['Province/State'],df['Province_State'])

df['Latitude']=np.where(pd.isnull(df['Latitude']),df['Lat'],df['Latitude'])
df['Longitude']=np.where(pd.isnull(df['Longitude']),df['Long_'],df['Longitude'])

df['Case_Fatality_Ratio']=np.where(pd.isnull(df['Case_Fatality_Ratio']),df['Case-Fatality_Ratio'],df['Case_Fatality_Ratio'])
df['Case_Fatality_Ratio']=np.where(df['Deaths']==0,0,df['Case_Fatality_Ratio'])

df['Incidence_Rate']=np.where(pd.isnull(df['Incidence_Rate']),df['Incident_Rate'],df['Incidence_Rate'])

df['Last_Update']=np.where(pd.isnull(df['Last_Update']),df['Last Update'],df['Last_Update'])


df.drop(columns=['FIPS','Admin2','Country/Region','Province/State','Lat','Long_','Case-Fatality_Ratio','Incident_Rate','Last Update'], axis='columns',inplace=True)

In [ ]:
df=df.assign(Population=lambda df: (df.Confirmed/df.Incidence_Rate)*100000)

In [ ]:
df

,Province_State,Country_Region,Last_Update,Confirmed,Deaths,Recovered,Active,Combined_Key,Case_Fatality_Ratio,Date,Incidence_Rate,Latitude,Longitude,Population
0,NaN,Afghanistan,2020-12-24 05:23:04,50433.0,2117.0,39692.0,8624.0,Afghanistan,4.197648,2020-12-23,129.553427,33.93911,67.709953,38928341.0
1,NaN,Albania,2020-12-24 05:23:04,54317.0,1117.0,29799.0,23401.0,Albania,2.056446,2020-12-23,1887.448746,41.15330,20.168300,2877800.0
2,NaN,Algeria,2020-12-24 05:23:04,96549.0,2696.0,64401.0,29452.0,Algeria,2.792364,2020-12-23,220.174923,28.03390,1.659600,43851043.0
3,NaN,Andorra,2020-12-24 05:23:04,7669.0,82.0,7106.0,481.0,Andorra,1.069240,2020-12-23,9925.580793,42.50630,1.521800,77265.0
4,NaN,Angola,2020-12-24 05:23:04,16931.0,393.0,9729.0,6809.0,Angola,2.321186,2020-12-23,51.514824,-11.20270,17.873900,32866268.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299,NaN,Jersey,2020-03-17T18:33:03,0.0,0.0,0.0,NaN,NaN,0.000000,2020-03-21,NaN,49.19000,-2.110000,NaN
300,NaN,Puerto Rico,2020-03-17T16:13:14,0.0,0.0,0.0,NaN,NaN,0.000000,2020-03-21,NaN,18.20000,-66.500000,NaN
301,NaN,Republic of the Congo,2020-03-17T21:33:03,0.0,0.0,0.0,NaN,NaN,0.000000,2020-03-21,NaN,-1.44000,15.556000,NaN
302,NaN,The Bahamas,2020-03-19T12:13:38,0.0,0.0,0.0,NaN,NaN,0.000000,2020-03-21,NaN,24.25000,-76.000000,NaN


In [ ]:
df[pd.isnull(df.Population)].sort_values(['Date']).tail(100)

,Province_State,Country_Region,Last_Update,Confirmed,Deaths,Recovered,Active,Combined_Key,Case_Fatality_Ratio,Date,Incidence_Rate,Latitude,Longitude,Population
1216,Hawaii,US,2020-12-21 13:27:30,286.0,2.0,0.0,284.0,"Out of HI, Hawaii, US",0.699301,2021-01-10,NaN,NaN,NaN,NaN
1210,Grand Princess,US,2020-08-04 02:27:56,103.0,3.0,0.0,100.0,"Grand Princess, US",2.912621,2021-01-10,NaN,NaN,NaN,NaN
1260,Idaho,US,2021-01-11 05:21:50,0.0,0.0,0.0,0.0,"Unassigned, Idaho, US",0.000000,2021-01-10,NaN,NaN,NaN,NaN
1158,Georgia,US,2021-01-11 05:21:50,27466.0,361.0,0.0,27105.0,"Out of GA, Georgia, US",1.314352,2021-01-10,NaN,NaN,NaN,NaN
1043,Florida,US,2021-01-11 05:21:50,3170.0,0.0,0.0,3170.0,"Unassigned, Florida, US",0.000000,2021-01-10,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
743,Alaska,US,2021-01-12 05:22:15,211.0,0.0,0.0,211.0,"Unassigned, Alaska, US",0.000000,2021-01-11,NaN,NaN,NaN,NaN
723,Alaska,US,2021-01-12 05:22:15,0.0,0.0,0.0,0.0,"Bristol Bay, Alaska, US",0.000000,2021-01-11,0.0,58.74514,-156.701064,NaN
702,Alabama,US,2020-12-21 13:27:30,0.0,0.0,0.0,0.0,"Out of AL, Alabama, US",0.000000,2021-01-11,NaN,NaN,NaN,NaN
1354,Illinois,US,2021-01-12 05:22:15,95.0,0.0,0.0,95.0,"Unassigned, Illinois, US",0.000000,2021-01-11,NaN,NaN,NaN,NaN


In [ ]:
alt.Chart(df[(df.Combined_Key=='Catalonia, Spain') | (df.Combined_Key=='Aragon, Spain') | (df.Combined_Key=='Madrid, Spain')]).mark_line().encode(x='Date:T',y='Deaths',color='Combined_Key')

alt.Chart(...)

In [ ]:
df[df.Country_Region=='Spain']

,Province_State,Country_Region,Last_Update,Confirmed,Deaths,Recovered,Active,Combined_Key,Case_Fatality_Ratio,Date,Incidence_Rate,Latitude,Longitude,Population
594,Andalusia,Spain,2020-12-24 05:23:04,256664.0,4912.0,10671.0,241081.0,"Andalusia, Spain",1.913786,2020-12-23,3045.587580,37.544300,-4.727800,8427405.0
595,Aragon,Spain,2020-12-24 05:23:04,77980.0,2577.0,3772.0,71631.0,"Aragon, Spain",3.304694,2020-12-23,5904.954316,41.597600,-0.905700,1320586.0
596,Asturias,Spain,2020-12-24 05:23:04,26469.0,1310.0,1063.0,24096.0,"Asturias, Spain",4.949186,2020-12-23,2589.402321,43.361400,-5.859300,1022205.0
597,Baleares,Spain,2020-12-24 05:23:04,31792.0,447.0,1533.0,29812.0,"Baleares, Spain",1.406014,2020-12-23,2675.598795,39.710358,2.995148,1188220.0
598,C. Valenciana,Spain,2020-12-24 05:23:04,134414.0,2756.0,9970.0,121688.0,"C. Valenciana, Spain",2.050382,2020-12-23,2701.805780,39.484000,-0.753300,4974969.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
597,Navarra,Spain,2020-10-23 04:24:46,27504.0,666.0,3905.0,22933.0,"Navarra, Spain",2.421466,2020-10-22,4231.736175,42.695400,-1.676100,649946.0
598,Pais Vasco,Spain,2020-10-23 04:24:46,56566.0,2029.0,16160.0,38377.0,"Pais Vasco, Spain",3.586960,2020-10-22,2597.296453,42.989600,-2.618900,2177880.0
599,Unknown,Spain,2020-10-23 04:24:46,0.0,0.0,0.0,0.0,"Unknown, Spain",0.000000,2020-10-22,NaN,NaN,NaN,NaN
3415,NaN,Spain,3/30/20 22:52,87956.0,7716.0,16780.0,63460.0,Spain,NaN,2020-03-30,NaN,40.463667,-3.749220,NaN
